In [1]:
# fix the cross-validation process using Pipeline
from sklearn.pipeline import make_pipeline
from __future__ import division # ensure that all division is float division
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
#import all the necessary modules
import os, sys, re
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()
sns.set_style("whitegrid")

In [2]:
os.chdir(r"C:\Users\pysouth\general_assembly\real_folder\Projects\data")
df=pd.read_csv("tweets_sample.csv")
df.Support.value_counts()
# map the label values for scikit-learn
df["Label"]=df.Support.map({"Neutral":0,"Panthers":1,"Broncos":2})
#determine feature and label variables
X=df.text
y=df.Label
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

# Classifiers

In [3]:
#multinomialNB classifier
from sklearn.naive_bayes import MultinomialNB
clfNB=MultinomialNB()
#knn classifier
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
#linear SVM
from sklearn import svm
linear_svc=svm.SVC(kernel='linear')
#RBF SVM
rbf_svm=svm.SVC(kernel='rbf',C=470)
#Logistic Regression
from sklearn import linear_model
logreg = linear_model.LogisticRegression(C=1e5)

In [4]:
vect=CountVectorizer()

In [5]:
from sklearn.cross_validation import cross_val_score
neighbors_range = range(1, 21)
max_grid=dict(max__feature=range(1,10))
param_grid = dict(kneighborsclassifier__n_neighbors=neighbors_range)
neighbors_range = range(1, 21)
pipe = make_pipeline(CountVectorizer(), KNeighborsClassifier())

In [6]:
cross_val_score(pipe, X, y, cv=5, scoring='accuracy').mean()

0.78129922629879223

In [7]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', KNeighborsClassifier()),
])


In [13]:
parameters = [{
    'vect__max_df': (0.5, 0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    #'tfidf__use_idf': (True, False),
    #'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': (0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet'),
    #'clf__n_iter': (10, 50, 80),
}]

In [9]:
from sklearn.grid_search import GridSearchCV
from __future__ import print_function

from pprint import pprint
from time import time
import logging


In [16]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(CountVectorizer(),TfidfTransformer(),KNeighborsClassifier())
grid = GridSearchCV(pipe, parameters, cv=5, scoring='accuracy')

TypeError: __init__() got an unexpected keyword argument 'ngram_range'

In [17]:
grid.fit(X_train, y_train)


ValueError: Invalid parameter vect for estimator Pipeline(steps=[('countvectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,...owski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'))]). Check the list of available parameters with `estimator.get_params().keys()`.

In [ ]:
print grid.best_score_
print grid.best_params_

In [ ]:

if __name__ == "__main__":
    # multiprocessing requires the fork to happen in a __main__ protected
    # block

    # find the best parameters for both the feature extraction and the
    # classifier
    grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

    print("Performing grid search...")
    print("pipeline:", [name for name, _ in pipeline.steps])
    print("parameters:")
    pprint(parameters)
    t0 = time()
    grid_search.fit(X_train, y_train)
    print("done in %0.3fs" % (time() - t0))
    print()

    print("Best score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
